# Matching Department of Health data (2011) to M&W

Import Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re #regular expressions 
from fuzzywuzzy import process, fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os 
import glob

###### Read in Data

                                                           M&W Data

In [2]:
pd.set_option('display.max_rows', None)
redFood = pd.read_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Data\Processed\ReducedwithWeights\dataproduct.csv", index_col=0)
redFood.rename(columns={'Super Group':'SuperGroup'})
redFood['Food Name'] = redFood['Food Name'].str.lower()
redFood.head()

,Food Code,Food Name,Group,Super Group,Sale format(s),Source,Similarity Score,Weight,PurEqualCon,Portion Consumed
0,17-208,"beer, bitter, best, premium, 1 pint",QA,Alcoholic beverages,pint,NaN,0.828152,574g,Y,1 pint
1,NaN,"beer, bitter, best, premium, bottle",QA,Alcoholic beverages,beer bottle,NaN,0.792934,250/300g,Y,bottle
2,NaN,"beer, bitter, best, premium, small can",QA,Alcoholic beverages,small can,NaN,0.782098,333g,Y,small can
3,NaN,"beer, bitter, best, premium, large can",QA,Alcoholic beverages,large can,NaN,0.785473,444g,Y,large can
4,17-224,cider,QC,Alcoholic beverages,"can, can multipack, bottle, bottle multipack",NaN,0.000000,NaN,NaN,NaN


                                                Fruit and Vegetables

In [3]:
fv = redFood[redFood['Super Group'].str.contains('Fruit|Vegetables')]
fv['Super Group'] = fv['Super Group'].fillna('')
fv['Weight'] = fv['Weight'].fillna('')
fv['PurEqualCon'] = fv['PurEqualCon'].fillna('')
fv['Portion Consumed'] = fv['Portion Consumed'].fillna('')

C:\Users\medekar\AppData\Local\Temp\ipykernel_27236\4286489466.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fv['Super Group'] = fv['Super Group'].fillna('')
C:\Users\medekar\AppData\Local\Temp\ipykernel_27236\4286489466.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fv['Weight'] = fv['Weight'].fillna('')
C:\Users\medekar\AppData\Local\Temp\ipykernel_27236\4286489466.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

                                                Department of Health Data

Converted Text data to a table

In [4]:
doh = pd.read_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Data\Processed\WeightData_sourced\DofHealth_FV.csv", index_col=0)
doh['Composite Sample Name'] = doh['Composite Sample Name'].str.lower()
doh[doh['Composite Sample Name'].str.contains('orange')]#.head()
doh['Sub-sample Number'] = doh['Sub-sample Number'].astype(str)

                                    Updating DoH text table with Sample weights tables

In [5]:
# Set the directory path where CSV files are located
directory_path = r'C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights'

# Use glob to get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(directory_path, '*.csv'))

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each CSV file, read it using pandas, and append to the list
for csv_file in csv_files:
    # Read CSV file into a DataFrame
    dataframe = pd.read_csv(csv_file)
    
    # Append DataFrame to the list
    dataframes.append(dataframe)

# Concatenate all DataFrames into a single DataFrame
dataframefull = pd.concat(dataframes, ignore_index=True)

Test

In [6]:
dataframefull.drop(['Unnamed: 0'], axis=1, inplace=True)
dataframefull['Composite Sample Name'] = dataframefull['Composite Sample Name'].str.lower()

In [7]:
dataframefull.rename(columns={'Sub-sample Number':'Sub_sampleNumber'}, inplace = True)
dataframefull['Sub_sampleNumber'] = dataframefull['Sub_sampleNumber'].astype(str)

In [8]:
# Create a dictionary to store sample weights from dataframefull
sample_weights = dict(zip(dataframefull['Sub_sampleNumber'], dataframefull['Sample Weight']))

In [15]:
#sample_weights

In [10]:
# Function to get the sample weight based on common number
def get_sample_weight(Sub_sampleNumber):
    return sample_weights.get(Sub_sampleNumber, '')

# Apply the function to calculate and add sample weights to the doh dataframe
doh['Sample Weight'] = doh['Sub-sample Number'].apply(get_sample_weight)

In [16]:
doh[doh['Sample Weight'] != ''].head()

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage,Sample Weight
0,1.0,"potatoes, old, raw , flesh only",1.1,"Tesco , Eastern Counties , white potatoes",Tesco,2.5kg,03.10.2011,"Tesco, Harford Bridges, Norwich",05-Oct,NaN,NaN,Plastic,Ambient,Ambient,136.4
1,1.0,"potatoes, old, raw, flesh only",nan,"Tesco , Eastern Counties , Desiree potatoes",Tesco,2.5kg,03.10.2011,"Tesco, Harford Bridges, Norwich",06-Oct,NaN,GV1 11:58 L2,Plastic,Ambient,Ambient,3 pack
2,1.0,"potatoes, old, raw, flesh only",nan,"Asda , Maris Piper p otatoes",Asda,2.5kg,03.10.2011,"Asda, Norwich",04-Oct,07-Oct,QVL43021C39176879 Site code: 790317,Plastic,Ambient,Ambient,3 pack
3,NaN,"potatoes, old, raw, flesh only",1.4,"Asda , Desiree p otatoes",Asda,2.5kg,03.10.2011,"Asda, Norwich",05-Oct,08-Oct,QVL 10906E39176268 site code: 790195,Plastic,Ambient,Ambient,117.0
4,1.0,"potatoes, old, raw, flesh only",1.5,"Sainsbury’s , British King Edward potatoes",Sainsbury’s,2.5kg,03.10.2011,"Sainsbury’s, Longwater , Norwich",05-Oct,07-Oct,A0283 B Line D 5 2001,Plastic,Ambient,Ambient,127.9


                                                    Matching M&W and DoH

In [12]:
%%time

def fuzzy_matching_score(source, target):
    return fuzz.partial_token_set_ratio(source, target)

def get_most_similar_indices(tfidf_red_food, tfidf_clean_retail):
    similarities = cosine_similarity(tfidf_red_food, tfidf_clean_retail)
    return similarities.argmax(axis=1)

def hybrid_fuzzy_matching(doh, fv):
    # Set thresholds for matching
    partial_token_set_ratio_threshold = 100
    cosine_similarity_threshold = 0.80
    min_similarity_threshold = 0.50

    
    # Step 1: Partial Token Set Ratio (Fuzzy Matching)
    doh['Partial Token Set Ratio'] = doh['Composite Sample Name'].apply(lambda x: fuzzy_matching_score(x, fv['Food Name']))
    potential_matches = doh[doh['Partial Token Set Ratio'] >= partial_token_set_ratio_threshold]
    
    print('Completed 1')

    # Step 2: TF-IDF and Cosine Similarity (Text Matching) - Use combined matches as input
    vectorizer = TfidfVectorizer()
    vectorizer.fit(potential_matches['Composite Sample Name'])
    tfidf_clean_doh = vectorizer.transform(potential_matches['Composite Sample Name'])
    tfidf_red_food = vectorizer.transform(fv['Food Name'])

    # Find the most similar match for each redFood product
    max_sim_indices = get_most_similar_indices(tfidf_red_food, tfidf_clean_doh)
    max_sim_scores = cosine_similarity(tfidf_red_food, tfidf_clean_doh).max(axis=1)
    
    print('Completed 2')

    # Update redFood DataFrame with matched information and sources
    fv['Similarity Score DoH'] = max_sim_scores
    fv['Weight DoH'] = potential_matches['Sample Weight'].values[max_sim_indices]
    fv['Pack Size'] = potential_matches['Pack size'].values[max_sim_indices]
    fv['Packaging DoH'] = potential_matches['Packaging'].values[max_sim_indices]
    fv['Purchase Date DoH'] = potential_matches['Purchase Date'].values[max_sim_indices]
    
    
    print('Completed 3')

    # Set to NaN for rows where similarity score is below the minimum threshold
    fv.loc[fv['Similarity Score DoH'] < min_similarity_threshold, ['Weight DoH', 'Packaging DoH', 'Pack Size', 'Purchase Date DoH']] = ''

    return fv

# Call the hybrid_fuzzy_matching function and get the matched DataFrame
redFood_matched = hybrid_fuzzy_matching(doh.copy(), fv.copy())

Completed 1
Completed 2
Completed 3
CPU times: total: 1.44 s
Wall time: 1.56 s


In [17]:
redFood_matched.head()

,Food Code,Food Name,Group,Super Group,Sale format(s),Source,Similarity Score,Weight,PurEqualCon,Portion Consumed,Similarity Score DoH,Weight DoH,Pack Size,Packaging DoH,Purchase Date DoH
155,14-319,"apple, with core,",F,Fruit,flesh and skin,NaN,0.529331,,,,0.412233,,,,
156,14-001,amla,FA,Fruit,NaN,NaN,0.000000,,,,0.000000,,,,
157,NaN,"apricots, canned",FA,Fruit,punnet,NaN,0.488128,,,,0.474719,,,,
158,14-040,"avocado pear, fuerte",FA,Fruit,"loose, individual",NaN,0.924432,75g,Y,average 1/2 pear,0.000000,,,,
159,14-387,"avocado pear, hass",FA,Fruit,"loose, individual",NaN,0.924432,75g,Y,average 1/2 pear,0.000000,,,,


Matched DoH data doesn't have good weight data. I am going to go back and look at the PdF to try find better information.


Tables at the end of pdf have weight info for all samples used. Get this into text table and try matching again